Kaggle - LANL Earthquake Prediction

General information Forecasting earthquakes is one of the most important problems in Earth science because of their devastating consequences. Current scientific studies related to earthquake forecasting focus on three key points: when the event will occur, where it will occur, and how large it will be. In this competition we try to predict time left to the next laboratory earthquake based on seismic signal data to answer the question of when earthquake will occur.

Training data represents one huge signal, but in test data we have many separate chunks, for each of which we need to predict time to failure.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as colors
# spectrogram with scipy
from scipy import signal

from tqdm import tqdm_notebook as tqdm
import gc

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./data"))

# Any results you write to the current directory are saved as output.

#garbage collect
gc.collect()

['train', 'sample_submission.csv', 'test', 'train.csv']


4

The data is huge, training data contains nearly 600 million rows and that is A LOT of data to understand.

In [3]:
%%time
# limit the load to 10M lines for display purpose
# train = pd.read_csv('./data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}, nrows=int(10e6)
# train = pd.read_csv('./data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}, nrows=int(50e6))
train = pd.read_csv('./data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
#Collect garbage
gc.collect()

CPU times: user 1min 7s, sys: 5.61 s, total: 1min 13s
Wall time: 1min 8s


In [7]:
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))
print("number of segments:",segments)

# X = pd.DataFrame(index=range(segments), dtype=np.float64,
#                        columns=features)
# Y = pd.DataFrame(index=range(segments), dtype=np.float64,
#                        columns=['time_to_failure'])

for segment in tqdm(range(segments)):
        
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values
    #print(x,y)

    fs = 1
    # f, t, Sxx = signal.spectrogram(partial_train.values[:,0], fs)
    # f, t, Sxx = signal.spectrogram(train.values[:,0], fs)
    f, t, Sxx = signal.spectrogram(x, # array_like - Time series of measurement values
                               fs = fs, # float, optional - Sampling frequency of the x time series. Defaults to 1.0.
                               window = ('tukey', 0.25), # str or tuple or array_like, optional - Desired window to use. If window is a string or tuple, it is passed to get_window to generate the window values, which are DFT-even by default. See get_window for a list of windows and required parameters. If window is array_like it will be used directly as the window and its length must be nperseg. Defaults to a Tukey window with shape parameter of 0.25.
                               nperseg = 447, # int, optional - Length of each segment. Defaults to None, but if window is str or tuple, is set to 256, and if window is array_like, is set to the length of the window
                               noverlap = 113, # int, optional - Number of points to overlap between segments. If None, noverlap = nperseg // 8. Defaults to None.
                               nfft = None, # int, optional - Length of the FFT used, if a zero padded FFT is desired. If None, the FFT length is nperseg. Defaults to None.
                               detrend = 'constant', # str or function or False, optional - Specifies how to detrend each segment. If detrend is a string, it is passed as the type argument to the detrend function. If it is a function, it takes a segment and returns a detrended segment. If detrend is False, no detrending is done. Defaults to ‘constant’.
                               return_onesided = True, # bool, optional - If True, return a one-sided spectrum for real data. If False return a two-sided spectrum. Note that for complex data, a two-sided spectrum is always returned
                               scaling = 'density', # { ‘density’, ‘spectrum’ }, optional - Selects between computing the power spectral density (‘density’) where Sxx has units of V**2/Hz and computing the power spectrum (‘spectrum’) where Sxx has units of V**2, if x is measured in V and fs is measured in Hz. Defaults to ‘density’
                               axis = -1, # int, optional - Axis along which the spectrogram is computed; the default is over the last axis (i.e. axis=-1)
                               mode = 'psd' # str, optional - Defines what kind of return values are expected. Options are [‘psd’, ‘complex’, ‘magnitude’, ‘angle’, ‘phase’]. ‘complex’ is equivalent to the output of stft with no padding or boundary extension. ‘magnitude’ returns the absolute magnitude of the STFT. ‘angle’ and ‘phase’ return the complex angle of the STFT, with and without unwrapping, respectively
                              )
    # Returns:
    # f : ndarray - Array of sample frequencies.
    # t : ndarray - Array of segment times.
    # Sxx : ndarray - Spectrogram of x. By default, the last axis of Sxx corresponds to the segment times.

    logsxx = np.log(Sxx)
    y = y[list(map(int, t))]  # Selects every given resized raw starting from 0
    
    # print(y)
    # break
    found = False
    index_found = -1
    for i in range(len(y)):
        if y[i] < 1e-3:
            index_found = i
            found = True
            break
        

    #if found is False: continue
        
    # print('x.shape:',x.shape)
    # print('y.shape:',y.shape)
    # print('t.shape:',t.shape)
    # print('f.shape:',f.shape)
    # # logsxx_transpose = list(map(list, zip(*logsxx)))
    # print('logsxx.shape:',logsxx.shape)
    # # print('x',x)
    # # print('y raw',y)
    # # print(t)
    # # print(logsxx_transpose)
    
    # # print('list(t)',list(map(int, t)))
    # print('y.shape filtered',y.shape)
    
    # print(logsxx)
    # https://www.infobyip.com/detectmonitordpi.php
    my_dpi = 120
    # print('t.shape[0]',t.shape[0])
    fig = plt.figure(figsize=(t.shape[0]/my_dpi, f.shape[0]/my_dpi), dpi=my_dpi, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.yscale('log')

    # plt.pcolormesh(t, f, logsxx, norm = colors.LogNorm(vmin=logsxx.min(), vmax=logsxx.max()))
    plt.pcolormesh(t, f, logsxx)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.ylim(0.01, 0.5)
    # cbar = plt.colorbar()
    # cbar.solids.set_edgecolor("face")
    #plt.draw()
    plt.savefig('./data/train/seg'+str(segment)+'-x'+str(index_found)+'.png', dpi=my_dpi)
    # plt.show()
    plt.close(fig)
    
    
    # figure, axes1 = plt.subplots(figsize=(15,6))

    # plt.title("Seismic Data Spectrogram for segment "+str(segment))
    # plt.yscale('log')
    
    # plt.pcolormesh(t, f, logsxx)
    # axes1.set_ylabel('Frequency [Hz]')
    # plt.xlabel('Time [sec]')
    # plt.legend(['Acoustic Data'])
    # plt.ylim(0.01, 0.5)
    
    # axes2 = axes1.twinx()
    # plt.plot(t,y, color='r')
    # axes2.set_ylabel('Time to Failure', color='r')
    # plt.legend(['Time to Failure'])

    # plt.savefig('./data/train/plot_seg'+str(segment)+'-x'+str(index_found)+'.png', dpi=my_dpi)
    # # plt.show()
    
    np.savetxt('./data/train/seg'+str(segment)+'.csv', y, delimiter=",", fmt='%s')
    
    # break


number of segments: 4194
